In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"

import argparse
import torch
import torchvision
import torch.nn.functional as F
import numpy as np

from nn.enums import ExplainingMethod
from nn.networks import ExplainableNet
from nn.utils import get_expl, plot_overview, clamp, load_image, make_dir
import keras
from keras import datasets

no display found. Using non-interactive Agg backend


Using TensorFlow backend.
/home/raj_vardhan/anaconda3/envs/torch_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/raj_vardhan/anaconda3/envs/torch_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/raj_vardhan/anaconda3/envs/torch_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [2]:
def get_beta(i, num_iter):
    """
    Helper method for beta growth
    """
    start_beta, end_beta = 10.0, 100.0
    return start_beta * (end_beta / start_beta) ** (i / num_iter)

def np_img_to_tensor(grayscale_img):
    rgb_img = np.repeat(grayscale_img[..., np.newaxis], 3, -1)
    im = Image.fromarray(rgb_img)
    x = torchvision.transforms.Normalize(mean=data_mean, std=data_std)(torchvision.transforms.ToTensor()(torchvision.transforms.Resize(224)(im)))
    x = x.unsqueeze(0).to(device)
    return x

In [3]:
args_cuda = True

# choices:['lrp', 'guided_backprop', 'gradient', 'integrated_grad','pattern_attribution', 'grad_times_input']
args_method = 'lrp'
args_beta_growth = None
args_img = '../data/collie4.jpeg'
args_target_img = '../data/tiger_cat.jpeg'
args_num_iter = 1500
args_prefactors = [1e11, 1e6]
args_lr = 0.0002
args_output_dir = '../output/'
args_role = 'defender'
args_dataset = 'fmnist'
args_root_dir = '../data/'

In [4]:
# options
device = torch.device("cuda" if args_cuda else "cpu")
method = getattr(ExplainingMethod, args_method)
print('Explanation method {} will be used'.format(method))

# load model
data_mean = np.array([0.0, 0.0, 0.0])
data_std = np.array([1.0, 1.0, 1.0])

vgg_model = torchvision.models.vgg16(pretrained=True)
model = ExplainableNet(vgg_model, data_mean=data_mean, data_std=data_std, beta=1000 if args_beta_growth else None)
if method == ExplainingMethod.pattern_attribution:
    model.load_state_dict(torch.load('../models/model_vgg16_pattern_small.pth'), strict=False)
model = model.eval().to(device)

#print(vgg_model)

Explanation method ExplainingMethod.lrp will be used


In [104]:
(x1, _) ,(_, _) = keras.datasets.fashion_mnist.load_data()
x2 = np.load(adv_dir+'/x_adv_ar.npy')
x2 = (x2*255).astype(np.uint8)


In [93]:
gs1 = x1[0]
rgb_img = np.repeat(gs1[..., np.newaxis], 3, -1)
im = Image.fromarray(rgb_img)

In [105]:
gs2 = x2[0]
rgb_img2 = np.repeat(gs2[..., np.newaxis], 3, -1)
im2 = Image.fromarray(rgb_img2)

In [100]:
np.max(gs1)

255

In [101]:
np.max(gs2)

254

In [81]:
gs2.shape

(28, 28)

In [106]:
import random
 
input_is_adv = True
if not input_is_adv:
    (x_train, y_train) ,(x_test, y_test) = datasets.fashion_mnist.load_data()
    # print(x_train.shape) ##(60000, 28, 28)
    args_class_idx = 6
    indices = np.where(y_train==args_class_idx)[0]
    
else:
    adv_dir = '../../xai-adv/data/postndss/defender/fmnist/target_next/cwl2/conf_0/'
    x_train = np.load(adv_dir+'/x_adv_ar.npy')
    x_train = (x_train*255).astype(np.uint8)
    indices = np.array([i for i in range(x_train.shape[0])])

from torchvision import datasets, transforms
from PIL import Image

num_samples = indices.shape[0]
# expls will store explanations for all the samples
expls = np.zeros((num_samples,28,28))

for i,idx in enumerate(indices):
    if (i+1)%100 == 0:
        print('Running for sample {}/{}'.format(i+1,num_samples))
    x = np_img_to_tensor(x_train[idx])
    x_adv = x.clone().detach().requires_grad_()
    
    # obtain the explanation
    if not input_is_adv:
        org_expl, org_acc, org_idx = get_expl(model, x, method)
        org_expl = org_expl.detach().cpu()
    else:
        desired_index = random.randint(0,1000)
        org_expl, org_acc, org_idx = get_expl(model, x, method, desired_index)
        org_expl = org_expl.detach().cpu()
    
    # convert explanation to numpy and subsequently downsize it to 28x28
    org_expl_np = org_expl.numpy()
    org_expl_np = org_expl_np.reshape(224, 224)
    im2 = Image.fromarray(org_expl_np)
    org_expl2 = torchvision.transforms.ToTensor()(torchvision.transforms.Resize(28)(im2))
    org_expl_np2 = org_expl2.numpy()
    org_expl_np2 = org_expl_np2.reshape(28,28)
    
    expls[i] = org_expl_np2


Running for sample 100/1000
Running for sample 200/1000
Running for sample 300/1000
Running for sample 400/1000
Running for sample 500/1000
Running for sample 600/1000
Running for sample 700/1000
Running for sample 800/1000
Running for sample 900/1000
Running for sample 1000/1000


In [5]:

    
# store the results
output_dir = args_root_dir + '/' + args_role + '/' + args_dataset + '/' + args_method + '/' + str(args_class_idx) + '/'

if not os.path.exists(output_dir):
    print('creating directory ',output_dir)
    os.makedirs(output_dir)

print('storing results in ',output_dir)
np.save(output_dir+'expls.npy', expls)

Using TensorFlow backend.
/home/raj_vardhan/anaconda3/envs/torch_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/raj_vardhan/anaconda3/envs/torch_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/raj_vardhan/anaconda3/envs/torch_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1